<a href="https://colab.research.google.com/github/ryanstwrt/osu-transport/blob/gh-pages/users/stewryan/Blackboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Blackboard System

Blackboard systems are used to solve large multi-facited problems in a piece-wise incremental manor. The basic components to a blackboard system can be seen in the bulleted list below.

*   Blackboard: Specialized space for agents to write their solutions to a problem too. This is typically stored in memory for quick access, but can also be written to a file for later use. The blackboard can contain specialized infrastructures which ease the reading and writing process for large problems.
*   Controller Agent: Initiates the problem, tells other agents when to perform their actions, and when  their results can be added to the blackboard.
*   Knowledge Agents: Contain specialized expertise in a problem and contribute to solving the problem at large with their specific piece of knowledge. For a multi-physics problem, this could be solving a set of equations with different methods or solving different sets of equation.

Below is a basic blackboard system which involved three agents; a controller and two knowledge agents.
The blackboard is used to create a HDF5 file, which the agents will write too, and keep track of the progress of the agents.
The simple example has a controller and two agents (A and B) writing the square of values 0 to 9 to the database.
Neither agent wants to copy what the other is dointn so they perform a simple check to ensure they are producing unique data. 
If so, they write the square of the given value to the database.
Agent A is working in forward from 0 to 9, while Agent B is working in reverse. 
At the end, the database has 10 entries ranging from 0 to 9.

In [1]:
pip install osbrain

     |████████████████████████████████| 92kB 5.8MB/s 
  Created wheel for osbrain: filename=osbrain-0.6.5-cp36-none-any.whl size=29223 sha256=b93f2bdee2bcd4619c82e9b92738b76a10005b470b4107918767faa6343cede6
  Stored in directory: /root/.cache/pip/wheels/cf/e2/0d/a21f92241a5836630a3f773468e2ebe7181016aa23fd5bbbcb
Successfully built osbrain


In [0]:
import osbrain
from osbrain import run_nameserver
from osbrain import run_agent
from osbrain import Agent
import time
import tables as tb
import os
import numpy as np

In [0]:
class Blackboard(object):
  """This class will be the blackboard from which knowledge agents write to and read from.
  The state variable keeps track of who is writting to the blackboard, and how many problems have been solved."""

  def __init__(self):
    self.agents = []
    self.state = {
        'problems': 0,
        'contributions': [],
    }
    #Set up the HDF5 databsase
    db = tb.open_file("blackboard_db", "w")
    db.close()

  def add_agent(self, agent):
    self.agents.append(agent)

class Controller(object):
  """Controls the flow of work distribution in the problem"""

  def __init__(self, blackboard):
    self.blackboard = blackboard

  def solve_problem(self):
    i = 0
    while self.blackboard.state['problems'] < 20:
      for agent in self.blackboard.agents:
        agent.contribute(i)
      i+=1        
    return self.blackboard.state['contributions']
  

class baseAgent(object):
  """Base agent to define __init__ and basic functions"""

  def __init__(self, blackboard, name):
    self.blackboard = blackboard
    self.name = name
    self.root = '/' + self.name
    db = tb.open_file("blackboard_db", "a")
    db.create_group(db.root, self.name)    
    db.close()

  def contribute(self, i):
    pass

  def squared(self, x):
    return x * x

class AgentA(baseAgent):
  """Agent A will count from 0 to 10"""

  def contribute(self, i):
    doubled = self.squared(i)
    with tb.open_file("blackboard_db", "a") as db:
      for node in db:
        if 'Val_{}'.format(str(i)) in node:
          self.blackboard.state['problems'] = 20
      else:
        db.create_array(self.root, 'Val_{}'.format(str(i)) , [doubled])
    self.blackboard.state['problems'] += 1
    self.blackboard.state['contributions'] += [self.__class__.__name__]

class AgentB(baseAgent):
  """Agent B will count in reverse from 10 to 0"""

  def contribute(self, i):
    i = 9 - i
    doubled = self.squared(i)
    with tb.open_file("blackboard_db", "a") as db:
      for node in db:
        if 'Val_{}'.format(str(i)) in node:
          self.blackboard.state['problems'] = 20
      else:
        db.create_array(self.root, 'Val_{}'.format(str(i)) , [doubled])
    self.blackboard.state['problems'] += 1
    self.blackboard.state['contributions'] += [self.__class__.__name__]

if __name__ == '__main__':
  os.system('rm blackboard_db')
  blackboard = Blackboard()
  blackboard.add_agent(AgentA(blackboard, 'AgentA'))
  blackboard.add_agent(AgentB(blackboard, 'AgentB'))
  c = Controller(blackboard)
  contributions = c.solve_problem()

  h5db = tb.open_file('blackboard_db', 'r+')
  print(h5db)
  h5db.close()

# Multi-Agent Systems

Multi-agent systems (MAS) focus on creating individual pieces of software (called agents) who perform some type of task independently of each other and of the user.
Agents can communicate with each other via messages to influence what the agents are working on.
A central theme for MAS is the autonomy of agents, where they can be designed to work cooperatively or competitively with other agents to accomplish goals.
The module [osBrain](https://osbrain.readthedocs.io/en/stable/) is used to create agents in the below examples and a view interactions methods are shown.
Some basic nomenclature is important to understand how MAS work in ```osBrain```.


*   Agent: Main entity which runs independently of other agents. Its major responsibilities are to poll for incomming messages, process the message, perform some type of action bas on the message, and repeat.
*   Proxy: Local objects which enables the user and other agents to access and communicate with a remote agent. Allows us to call methods or access attributes.



In [5]:
if __name__ == '__main__':
    # Create and server to house the multiple agents that will be created
    # This is where the agents will reside and how they can be communicated with
    ns = run_nameserver()
    #The run_agent() method creates a proxy to an agent on a remote server, where name='' allows users to create an alias for that proxy
    #Two methods to create an agent; the first creates an agent with no variables assigned to it, the second method creates a variable assiged proxy to an agent
    run_agent(name='Exp1')
    agent2 = run_agent(name='Exp2')

    #Create a proxy to agent Exp1 to easily access it
    agent1 = ns.proxy('Exp1')
    #Log a message via the proxy Exp1 agent
    agent1.log_info(' Hello World, I am tesing an agent')
    agent2.log_info('Hello, I am a second agent')

    #Determine what agents are alie in the server
    for alias in ns.agents():
      print(alias)
    #shutdown the server as it is no longer needed, otherwise the agents will continue to run
    ns.shutdown

Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:17425 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:17425
INFO [2019-11-01 12:57:34.563948] (Exp1):  Hello World, I am tesing an agent
INFO [2019-11-01 12:57:34.570184] (Exp2): Hello, I am a second agent
Exp1
Exp2


# Multi-Agent Blackboard System (MABS)
## Base Classes
The following section defines our base classes for the multi-agent blackboard system.
To reduce the number of communication lines, the controller agent from the previous sections has been incorporated with the blackboard class.
The blackboard class will now act as both the repository for information obtained by the agents, and will determine if/when agents should perform their next action.

Along with this, the agents no longer inheret from ```object```, but instead inheret from ```Agent```. 
```Agent``` is the base class for osBrain which creates a remote object that performs work independently of other agents.
These classes are interacted with via a proxy class, which is seen in the ```__main__``` section, where ```ka_a``` is a proxy which allows us to interact with ```KA_A```.

In [0]:
def log_message(self, message):
  self.log_info('Recieved: Running Simulation {}'.format(message))

def permission_granted(self, message):
  return message

class Blackboard(Agent):
  def on_init(self):
    self.bind('REP', alias='ask_permission', handler='ask_permission')
    self.bind('PUSH', alias='run_opt_A', handler='run_opt')
    self.agents = []
    self.state = {
        'problems': 0,
        'contributions': [],
    }
    db = tb.open_file("blackboard_db", "w")
    db.close()

  def solve_problem(self):
    i = 0
    while i < 2:
      for agent in self.agents:
        self.run_opt(agent, i)
      i+=1        
    return self.state['contributions']

  def add_agent(self, agent):
    self.agents.append(agent)

  def ask_permission(self, message):
    return message    
  
  def run_opt(self, agent, i):
    name = agent.get_attr('name')
    if name == 'ka_a':
      self.send('run_opt_A', i)
    elif name == 'ka_b':
      self.send('run_opt_B', i)


class baseKA(Agent):
  """Base agent to define __init__ and basic functions"""
  
  def on_init(self):
    self.connect(self.bb_addr(self.bb_perm_alias), alias=self.bb_perm_alias)
    # This connects to the blackboard. When the blackboard sends the agent a message, it will 
    # trigger the simulate method as the handler to solve the problem
    self.connect(self.bb_addr(self.bb_opt_alias), alias=self.bb_opt_alias, handler='simulate')
    self.root = '/{}'.format(self.name)
    self.db_entry = []
    self.doubled = 0
    db = tb.open_file("blackboard_db", "a")
    db.create_group(db.root, self.name)    
    db.close()

  def simulate(self, i):
    pass

  def write_to_db(self):
    pass

  def squared(self, x):
    return x * x

In [4]:
class KA_A(baseKA):

  def simulate(self, i):
    time.sleep(2)
    self.doubled = self.squared(i)
    print(self.doubled)

  def write_to_db(self):
    print('at writing step')
    with tb.open_file("blackboard_db", "a") as db:
      for node in db:
        if 'Val_{}'.format(self.doubled) in node:
          break
      else:
        db.create_array(self.root, 'Val_{}'.format(self.doubled), [self.doubled])
  
  def ask_permission_to_write(self):
    can_send = False
    i=0
    while can_send != True:
      if i > 3:
        can_send = True
      i += 1
      self.send(self.bb_perm_alias, can_send)
      can_send = self.recv(self.bb_perm_alias)
      if can_send == True:
        self.write_to_db()        
       
if __name__ == '__main__':
  os.system('rm blackboard_db')
  ns = run_nameserver()
  bb = run_agent(name='blackboard', base=Blackboard)
  ka_a = run_agent(name='ka_a', base=KA_A, attributes={'bb_addr':bb.addr,
                                                       'bb_perm_alias':'ask_permission',
                                                       'bb_opt_alias':'run_opt_A'})
  bb.add_agent(ka_a)

#  ka_a.ask_permission_to_write()

  contributions = bb.solve_problem()
  h5db = tb.open_file('blackboard_db', 'r+')
  print(h5db)
  h5db.close()
  ns.shutdown()

Broadcast server running on 0.0.0.0:9091
NS running on 127.0.0.1:16559 (127.0.0.1)
URI = PYRO:Pyro.NameServer@127.0.0.1:16559
blackboard_db (File) ''
Last modif.: 'Fri Nov  1 12:35:25 2019'
Object Tree: 
/ (RootGroup) ''
/ka_a (Group) ''

0
1
